In [1]:
import tensorflow as tf
import keras.layers as layers
import numpy as np

Using TensorFlow backend.


In [2]:
def shape2d(a):
    """
    Ensures a 2D shape.
    Args:
        a: a int or tuple/list of length 2
    Returns:
        list: of length 2. if ``a`` is a int, return ``[a, a]``.
    """
    if type(a) == int:
        return [a, a]
    if isinstance(a, (list, tuple)):
        assert len(a) == 2
        return list(a)
    raise RuntimeError("Illegal shape: {}".format(a))

In [3]:
def batch_norm_relu(in_tensor, name=None):
    """
    Shorthand for Batch Normalization + ReLU.
    Args:
        in_tensor: input tensor
        name: name or scope of the variable
    Returns:
        normalised plus relu activated tensor
    """
    if name is None:
        name = 'bn'
    else:
        name = name + '/bn'
        
    x = layers.BatchNormalization(axis=3, epsilon=1.0e-5, name=name)(in_tensor)
    return layers.Activation('relu', name='relu')(x)

In [4]:
from keras.layers import Input, Dense, Lambda, Concatenate
from keras.models import Model

In [5]:
def split_layers(in_tensor):
    weights = tf.get_variable(
                'kernel', kernel_size, dtype=inputs_dtype, initializer=kernel_initializer)
            
    # split into sub-tensors and perform group convolution
    in_tensor = tf.split(in_tensor, split, -1)
    #in_tensor = Lambda(lambda x: tf.split(x, split, -1))(in_tensor)
    weights_ = tf.split(weights, split, -1)
    #weights_ = Lambda(lambda x: tf.split(x, split, -1))(weights)
    
    outputs = [tf.nn.conv2d(i, k, strides=strides, padding=padding)
                       for i, k in zip(in_tensor, weights_)]
    x = tf.concat(outputs, -1)
    
    if use_bias:
            # add bias term
            biases = tf.get_variable(
                'bias', [kernel_size[-1]], dtype=inputs_dtype, initializer=bias_initializer)
            x = tf.nn.bias_add(x, biases)
    
    return x

In [23]:
bKerasLyr=True

def conv2d(
        name,
        in_tensor,
        ch,
        kernel_size,
        strides=1,
        padding='SAME',
        activation=None,
        use_bias=False,
        kernel_initializer=None,
        bias_initializer=tf.zeros_initializer(),
        split=1):
    """
    Perform 2D convolution operation
    Args:
        name: name or scope of the variable
        in_tensor: input tensor to convolution
        ch: number of channels in the input
        kernel_size: size of kernel. Assumes square kernel
        strides: number of pixels to translate kernel during convolution
        padding: 'SAME' or 'VALID' padding
        activation (string): activation function to apply to output of convolution
        use_bias: whether to add bias to output of convolution (Bool)
        kernel_initializer: method to initialise weights for convolution
        bias_initializer: method to initialise weights for bias term
        split: number of sub-tensors for group convolution (if 1, then normal convolution is used)
    Returns:
        Tensor output of 2d convolution, with optional bias and activation
    """
    if split == 1:
        # use tf.keras.layers.Conv2D is performing stanard convolution
        x = layers.Conv2D(ch, kernel_size, strides=strides, padding=padding, use_bias=use_bias, name=name)(in_tensor)
    else:
        with tf.variable_scope(name):
            # convert strides to the form used by tf.nn.conv2d
            if strides is None or strides == 1:
                strides = [1, 1, 1, 1]
            elif strides == 2:
                strides = [1, 2, 2, 1]

            # shape of the input tensor
            inputs_shape = in_tensor.get_shape().as_list()

            # convert kernel size to the form used by tf.nn.conv2d
            kernel_size = [kernel_size, kernel_size] + [int(inputs_shape[-1]/split), ch]
            inputs_dtype = in_tensor.dtype
            
            if bKerasLyr==False:
                weights = tf.get_variable(
                    'kernel', kernel_size, dtype=inputs_dtype, initializer=kernel_initializer)

                # split into sub-tensors and perform group convolution
                in_tensor = tf.split(in_tensor, split, -1)
                #in_tensor = Lambda(lambda x: tf.split(x, split, -1))(in_tensor)
                weights_ = tf.split(weights, split, -1)
                #weights_ = Lambda(lambda x: tf.split(x, split, -1))(weights)
                outputs = [tf.nn.conv2d(i, k, strides=strides, padding=padding)
                           for i, k in zip(in_tensor, weights_)]
                x = tf.concat(outputs, -1)
            else:
                x=Lambda(split_layers)(in_tensor)

        if use_bias:
            # add bias term
            biases = tf.get_variable(
                'bias', [kernel_size[-1]], dtype=inputs_dtype, initializer=bias_initializer)
            x = tf.nn.bias_add(x, biases)

    if activation is not None:
        # apply activation function
        if activation == 'bnrelu':
            x = batch_norm_relu(x, name=name)
        else:
            x = layers.Activation(activation)(x)
    return x

In [24]:
def upsample2x(in_tensor, shape=2, unpool_mat=np.ones((2, 2), dtype='float32')):
    """
    Up sample the input with a fixed matrix to perform kronecker product with.
    Args:
        in_tensor: input tensor
        shape: int or (h, w) tuple
        unpool_mat: a tf.Tensor or np.ndarray 2D matrix with size=shape.
                    If is None, will use a matrix with 1 at top-left corner
    Returns:
        2x UpSampled tensor via unpooling
    """

    input_shape = in_tensor.get_shape().as_list()
    shape = shape2d(shape)

    # check unpool_mat
    if unpool_mat is None:
        mat = np.zeros(shape, dtype='float32')
        mat[0][0] = 1
        unpool_mat = tf.constant(mat, name='unpool_mat')
    elif isinstance(unpool_mat, np.ndarray):
        unpool_mat = tf.constant(unpool_mat, name='unpool_mat')
    assert unpool_mat.shape.as_list() == list(shape)  # check as_list() works for tuple, I am getting a warning?

    x = tf.transpose(in_tensor, [0, 3, 1, 2])

    # perform a tensor-matrix kronecker product
    x = tf.expand_dims(x, -1)
    mat = tf.expand_dims(unpool_mat, 0)
    x = tf.tensordot(x, mat, axes=1)
    x = tf.transpose(x, [0, 2, 4, 3, 5, 1])

    return tf.reshape(x, [-1, shape[0]*input_shape[1], shape[1]*input_shape[2], input_shape[-1]])

In [25]:
def crop_op(x, cropping, data_format='channels_first'):
    """
    Center crop image
    Args:
        cropping is the substracted portion
    """
    crop_t = cropping[0] // 2
    crop_b = cropping[0] - crop_t
    crop_l = cropping[1] // 2
    crop_r = cropping[1] - crop_l
    if data_format == 'channels_first':
        x = x[:,:,crop_t:-crop_b,crop_l:-crop_r]
    else:
        x = x[:,crop_t:-crop_b,crop_l:-crop_r]
    return x

In [26]:
#tf.enable_eager_execution()

In [27]:
def res_blk(name, in_tensor, ch, kernel_size, count, strides=1):
    """
    Residual block consisting of <count> residual units.
    He, Kaiming, et al. "Deep residual learning for image recognition." 
    Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.
    Args:
        name: variable scope name
        in_tensor: input tensor
        ch: number of output channels
        kernel_size: kernel size
        count: number of residual units
        strides: strides of second convolution
    Returns:
        out_tensor: output of residual block
    """
    ch_in = in_tensor.get_shape().as_list()
    with tf.variable_scope(name):
        for i in range(0, count):
            with tf.variable_scope('block' + str(i)):
                x = in_tensor if i == 0 else batch_norm_relu(in_tensor, 'preact')
                x = conv2d('conv1', x, ch[0], kernel_size[0], activation='bnrelu')
                x = conv2d('conv2', x, ch[1], kernel_size[1],
                           strides=strides if i == 0 else 1, activation='bnrelu')
                x = conv2d('conv3', x, ch[2], kernel_size[2])
                if (strides != 1 or ch_in[-1] != ch[2]) and i == 0:
                    in_tensor = conv2d('convshortcut', in_tensor, ch[2], 1, strides=strides)
                in_tensor = in_tensor + x
        # end of each group need an extra activation
        out_tensor = batch_norm_relu(in_tensor, 'bnlast')
    return out_tensor
####


def dense_blk(name, in_tensor, ch, kernel_size, count, split, padding='VALID'): 
    """
    Dense block consisting of <count> dense units.
    Huang, Gao, et al. "Densely connected convolutional networks." 
    Proceedings of the IEEE conference on computer vision and pattern recognition. 2017.
    Args:
        name: variable scope name
        in_tensor: input tensor
        ch: number of channels
        kernel_size: kernel size
        count: number of dense units within block
        split: number of sub tensors if using group convolution
        padding: type of padding- choose 'SAME' or 'VALID'.
    Returns:
        out_tensor:
    """
    with tf.variable_scope(name):
        for i in range(0, count):
            with tf.variable_scope('blk/' + str(i)):
                x = batch_norm_relu(in_tensor, 'preact_bna')
                x = conv2d('conv1', x, ch[0], kernel_size[0],
                           padding=padding, activation='bnrelu')
                x = conv2d('conv2', x, ch[1], kernel_size[1], padding=padding, split=split)
                ##
                if padding == 'VALID':
                    x_shape = x.get_shape().as_list()
                    in_shape = in_tensor.get_shape().as_list()
                    in_tensor = crop_op(in_tensor, (in_shape[1] - x_shape[1], in_shape[2] - x_shape[2]))

                in_tensor = tf.concat([in_tensor, x], axis=-1)
        out_tensor = batch_norm_relu(in_tensor, 'blk_bna')
    return out_tensor
####


def encoder(in_tensor, input_pad):
    """
    Pre-activated ResNet50 Encoder
    Args:
        in_tensor: input tensor
        input_pad: type of padding for first convolution- 'SAME' or 'VALID'
    Returns:
        [d1, d2, d3, d4]: ResNet50 block outputs
    """
    d1 = conv2d('conv0', in_tensor, 64, 7, padding=input_pad, strides=1, activation='bnrelu')
    d1 = res_blk('group0', d1, [64,  64,   256], [1, 3, 1], 3, strides=1)
    d2 = res_blk('group1', d1, [128, 128,  512], [1, 3, 1], 4, strides=2)
    d3 = res_blk('group2', d2, [256, 256, 1024], [1, 3, 1], 6, strides=2)
    d4 = res_blk('group3', d3, [512, 512, 2048], [1, 3, 1], 3, strides=2)
    d4 = conv2d('conv_bot', d4, 1024, 1, padding='SAME')
    return [d1, d2, d3, d4]
####


def decoder(name, in_tensor, ksize):
    """
    Dense decoder unit. 
    Upsamples ResNet50 features by using a combination 
    of upsampling operations and dense blocks. 
    Args:
        name: variable scope name
        in_tensor: input tensor
        ksize: kernel size to use in decoder
    Returns:
        [u3, u2x, u1]:
    """
    pad = 'VALID'  # to prevent boundary artefacts
    with tf.variable_scope(name):
        with tf.variable_scope('u3'):
            u3 = Lambda(upsample2x)(in_tensor[-1])

            # skip connection
            u3_skip = in_tensor[-2]
            u3_sum = tf.add_n([u3, u3_skip])

            u3 = conv2d('conva', u3_sum, 256, ksize, strides=1, padding=pad)
            u3 = dense_blk('dense', u3, [128, 32], [1, ksize], 8, split=4, padding=pad)
            u3 = conv2d('convf', u3, 512, 1, strides=1)
        ####
        with tf.variable_scope('u2'):          
            u2 = Lambda(upsample2x)(u3)

            # skip connection
            u2_skip = in_tensor[-3]
            u2_shape = u2.get_shape().as_list()
            u2_skip_shape = u2_skip.get_shape().as_list()
            u2_sum = tf.add_n([u2, crop_op(
                u2_skip, (u2_skip_shape[1]-u2_shape[1], u2_skip_shape[2]-u2_shape[2]))])

            u2x = conv2d('conva', u2_sum, 128, ksize, strides=1, padding=pad)
            u2 = dense_blk('dense', u2x, [128, 32], [1, ksize], 4, split=4, padding=pad)
            u2 = conv2d('convf', u2, 256, 1, strides=1)
        ####
        with tf.variable_scope('u1'):          
            u1 = Lambda(upsample2x)(u2)

            # skip connection
            u1_skip = in_tensor[-4]
            u1_shape = u1.get_shape().as_list()
            u1_skip_shape = u1_skip.get_shape().as_list()
            u1_sum = tf.add_n([u1, crop_op(
                u1_skip, (u1_skip_shape[1]-u1_shape[1], u1_skip_shape[2]-u1_shape[2]))])

            u1 = conv2d('conva', u1_sum, 64, ksize, strides=1, padding='SAME')

    return [u3, u2x, u1]

In [28]:
def graph(network, is_training, images=None, decoder_ksize=5, num_of_classes=5, input_pad='VALID'):
    """
    Args:
        network: input Model class
        is_training: True if training the network otherwise False
        images: input images in NHWC format
        decoder_ksize: kernel size for decoders 
        num_of_classes: number of classes at output of NC branch
        input_pad: type of padding for first convolution ('SAME' or 'VALID')
    Returns:
        logi_nc: Nuclei classification logits
        logi_np: Nuclei Pixels logits
        logi_hv: Horizontal Vertical logits
    """
    print(network.batch_size)
    print(is_training)

    images = images/255.0  # normalise input between 0 and 1

    ####
    d = encoder(images, input_pad)

    ####
    np_feat = decoder('np', d, decoder_ksize)
    npx = batch_norm_relu(np_feat[-1], 'preact_out_np')

    hv_feat = decoder('hv', d, decoder_ksize)
    hv = batch_norm_relu(hv_feat[-1], 'preact_out_hv')

    tp_feat = decoder('tp', d, decoder_ksize)
    tp = batch_norm_relu(tp_feat[-1], 'preact_out_tp')

    # Nuclei Type Pixels (TP)
    logi_nc = conv2d('conv_out_tp', tp, num_of_classes, 1, use_bias=True)
    logi_nc = layers.Softmax(axis=-1, name='softmax')(logi_nc)

    # Nuclei Pixels (NP)
    logi_np = conv2d('conv_out_np', npx, 2, 1, use_bias=True)
    logi_np = layers.Softmax(axis=-1, name='softmax')(logi_np)

    # Horizontal-Vertical (HV)
    logi_hv = conv2d('conv_out_hv', hv, 2, 1, use_bias=True)

    return logi_nc, logi_np, logi_hv

In [29]:
from keras.models import Model

input_layer=layers.Input(shape=(270,270,3))

In [30]:
 d = encoder(input_layer, 'VALID')

In [31]:
decoder_ksize=5

np_feat = decoder('np', d, decoder_ksize)
npx = batch_norm_relu(np_feat[-1], 'preact_out_np')

hv_feat = decoder('hv', d, decoder_ksize)
hv = batch_norm_relu(hv_feat[-1], 'preact_out_hv')

tp_feat = decoder('tp', d, decoder_ksize)
tp = batch_norm_relu(tp_feat[-1], 'preact_out_tp')

# Nuclei Type Pixels (TP)
logi_nc = conv2d('conv_out_tp', tp, num_of_classes, 1, use_bias=True)
logi_nc = layers.Softmax(axis=-1, name='softmax')(logi_nc)

# Nuclei Pixels (NP)
logi_np = conv2d('conv_out_np', npx, 2, 1, use_bias=True)
logi_np = layers.Softmax(axis=-1, name='softmax')(logi_np)

# Horizontal-Vertical (HV)
logi_hv = conv2d('conv_out_hv', hv, 2, 1, use_bias=True)

NameError: name 'kernel_size' is not defined